In [ ]:
import numpy as np
import os
import cv2
from PIL import Image

In [ ]:
def preprocess_img(file):
    img = Image.open(file)
    img = img.resize((30,30))
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255
    return img

In [ ]:
def create_imgs_arr(path):
    array = []
    for root,dirs,files in os.walk(path):
        for file in files:
            array.append(preprocess_img(path+file))
    return array
        

In [ ]:
center = np.array([])
extreme = np.array([])

center = create_imgs_arr("./final_dataset/UnityEyes_Windows/cropped_center/")
extreme = create_imgs_arr("./final_dataset/UnityEyes_Windows/cropped_extreme/")

In [ ]:
center = np.array(center)
extreme = np.array(extreme)

In [ ]:
print(center.shape)
print(extreme.shape)

In [ ]:
trainx = np.vstack((center,extreme))
zeros = np.zeros((2387,1))
ones = np.ones((2168,1))
labels = np.vstack((zeros,ones))


In [ ]:
trainx = trainx.reshape(trainx.shape[0], 30, 30, 1)
print(trainx.shape)
print(labels.shape)

In [ ]:
from sklearn.utils import shuffle
trainx, labels = shuffle(trainx, labels)

In [ ]:
import keras
from keras.layers import *
from keras.models import Sequential
from keras import optimizers

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(30,30,1)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
hist = model.fit(trainx,labels,shuffle=True,batch_size=128,validation_split=0.1,epochs=30)

In [ ]:
model.save("eye_model.h5")

In [ ]:
from __future__ import division
import cv2
from mtcnn.mtcnn import MTCNN
from eye import Eye
from calibration import Calibration
import dlib
import os
import numpy as np
import json
import os
from keras.models import load_model
from PIL import Image

In [ ]:
calibration = Calibration()
_predictor = dlib.shape_predictor("trained_models/shape_predictor_68_face_landmarks.dat")
detector = MTCNN()
model_eye = load_model('./trained_models/eye_model.h5')

In [ ]:
def preprocess_img(img):
    img = Image.fromarray(img)
    img = img.resize((30,30))
    img = np.array(img)
    img = img/255
    img = img.reshape(30, 30, 1)
    img = np.expand_dims(img,axis=0)
    return img

In [ ]:
cam = cv2.VideoCapture(0)
while True:
    ret,frame = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(frame)
    for i in range(len(faces)) :
    
        bounding_box = faces[i]['box']
        x1, y1, width, height = bounding_box
#         face = [(x1,y1),(x1+width,y1+height)]
        face = dlib.rectangle(x1,y1,x1+width,y1+height)
        landmarks = _predictor(frame, face)
        left_eye = []
        right_eye = []
        left_eye.append([landmarks.part(36).x,landmarks.part(36).y])
        left_eye.append([landmarks.part(37).x,landmarks.part(37).y])
        left_eye.append([landmarks.part(38).x,landmarks.part(38).y])
        left_eye.append([landmarks.part(39).x,landmarks.part(39).y])
        left_eye.append([landmarks.part(40).x,landmarks.part(40).y])
        left_eye.append([landmarks.part(41).x,landmarks.part(41).y])
        
        right_eye.append([landmarks.part(42).x,landmarks.part(42).y])
        right_eye.append([landmarks.part(43).x,landmarks.part(43).y])
        right_eye.append([landmarks.part(44).x,landmarks.part(44).y])
        right_eye.append([landmarks.part(45).x,landmarks.part(45).y])
        right_eye.append([landmarks.part(46).x,landmarks.part(46).y])
        right_eye.append([landmarks.part(47).x,landmarks.part(47).y])
        
        left_eye = np.array(left_eye)
        right_eye = np.array(right_eye)
        
        rect_left = cv2.boundingRect(left_eye)
        x,y,w,h = rect_left
        txt_left = (x,y)
        croped_left = frame_g[y:y+h, x:x+w].copy()
        
        left_eye = left_eye - left_eye.min(axis=0)
        cv2.polylines(croped_left, np.array([left_eye], int), True, (0,0,0), 2)
        mask_left = np.zeros(croped_left.shape[:2], np.uint8)
        cv2.drawContours(mask_left, [left_eye], -1, (255, 0, 0), -1, cv2.LINE_AA)
        dst_left = cv2.bitwise_and(croped_left, croped_left, mask=mask_left)
#         cv2.imshow("left",dst_left)
        
        rect_right = cv2.boundingRect(right_eye)
        x,y,w,h = rect_right
        txt_right = (x,y)
        croped_right = frame_g[y:y+h, x:x+w].copy()
        right_eye = right_eye - right_eye.min(axis=0)
        cv2.polylines(croped_right, np.array([right_eye], int), True, (0,0,0), 2)
        mask_right = np.zeros(croped_right.shape[:2], np.uint8)
        cv2.drawContours(mask_right, [right_eye], -1, (255, 0, 0), -1, cv2.LINE_AA)
        dst_right = cv2.bitwise_and(croped_right, croped_right, mask=mask_right)
#         cv2.imshow("right",dst_right)
        left_eye_img = preprocess_img(dst_left)
        right_eye_img = preprocess_img(dst_right)
        
        left_pred = model_eye.predict(left_eye_img)
        right_pred = model_eye.predict(right_eye_img)
        print(left_pred,right_pred)
        if left_pred>0.5:
            cv2.putText(frame,"cheating",txt_left,cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame,"ok",txt_left,cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 2, cv2.LINE_AA)
        if right_pred>0.5:
            cv2.putText(frame,"cheating",txt_right,cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 2, cv2.LINE_AA)
        else:
            cv2.putText(frame,"ok",txt_right,cv2.FONT_HERSHEY_SIMPLEX, 1,(255,0,0), 2, cv2.LINE_AA)
        cv2.imshow("img", frame)
        
cam.release()
cv2.destroyAllWindows()

#         cv2.imwrite(dst_path+"/"+file.split('.')[0]+".jpg",dst)